<a href="https://colab.research.google.com/github/honganhss/OpenDataFitHou/blob/main/ParseRDF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install rdflib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 565.4/565.4 kB 7.9 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata

# Lấy giá trị secret
GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')

# Kiểm tra xem đã lấy được chưa
if GITHUB_TOKEN:
    print("✅ Đã lấy token, độ dài:", len(GITHUB_TOKEN))
else:
    print("❌ Không tìm thấy token")


✅ Đã lấy token, độ dài: 40


In [ ]:
import os
import requests
import base64
import json

GITHUB_USER = "honganhss"
GITHUB_REPO = "OpenDataFitHou"
FILE_PATH = "data/playground.geojson"
BRANCH = "main"
GITHUB_TOKEN = os.environ['GITHUB_TOKEN']

url = f"https://api.github.com/repos/{GITHUB_USER}/{GITHUB_REPO}/contents/{FILE_PATH}?ref={BRANCH}"

headers = {
    "Authorization": f"token {GITHUB_TOKEN}",
    "Accept": "application/vnd.github.v3+json"
}

response = requests.get(url, headers=headers)

if response.status_code == 200:
    data = response.json()
    file_content = base64.b64decode(data['content']).decode('utf-8')
    geojson_data = json.loads(file_content)
    print("✅ Lấy thành công, số feature:", len(geojson_data.get("features", [])))
else:
    print("❌ Lỗi:", response.status_code, response.text)


✅ Lấy thành công, số feature: 136


In [ ]:
from rdflib import Graph, Literal, RDF, URIRef, Namespace, RDFS

GEO = Namespace("http://www.opendatafithou.net/ont/geosparql#")
EX = Namespace("http://opendatafithou.org/playground/")

g = Graph()
g.bind("geo", GEO)
g.bind("ex", EX)

for feature in geojson_data.get("features", []):
    props = feature.get("properties", {})
    osm_id = props.get("osm_id")
    if not osm_id:
        continue
    subject = EX[str(osm_id)]

    name = props.get("name") or props.get("tags", {}).get("name")
    if name:
        g.add((subject, RDFS.label, Literal(name)))

    # Các tag khác
    tags = props.get("tags", {})
    for k, v in tags.items():
        if k != "name":
            g.add((subject, EX[k], Literal(v)))

    # Geometry
    geom = feature.get("geometry")
    if geom:
        geom_uri = URIRef(str(subject) + "/geometry")
        g.add((geom_uri, RDF.type, GEO[geom["type"]]))
        coords = geom["coordinates"]
        if geom["type"] == "Point":
            wkt = f"POINT({coords[0]} {coords[1]})"
        g.add((geom_uri, GEO.asWKT, Literal(wkt)))
        g.add((subject, GEO.hasGeometry, geom_uri))

g.serialize("playgrounds.ttl", format="turtle")
print("✅ Đã lưu RDF: playgrounds.ttl")


✅ Đã lưu RDF: playgrounds.ttl


In [ ]:
from rdflib import Graph

# Load file RDF vừa tạo
g = Graph()
g.parse("playgrounds.ttl", format="turtle")

# Hiển thị 10 triple đầu tiên
print("🔹 10 triple đầu tiên trong RDF:")
for i, (s, p, o) in enumerate(g):
    print(f"{i+1}. {s} -- {p} --> {o}")
    if i >= 9:
        break


🔹 10 triple đầu tiên trong RDF:
1. http://opendatafithou.org/playground/1228194746/geometry -- http://www.opendatafithou.net/ont/geosparql#asWKT --> POINT(105.7398883 21.0044387)
2. http://opendatafithou.org/playground/1196001335/geometry -- http://www.opendatafithou.net/ont/geosparql#asWKT --> POINT(105.7525519 21.0054693)
3. http://opendatafithou.org/playground/10765202472 -- http://opendatafithou.org/playground/website --> https://www.sanchoi.org/
4. http://opendatafithou.org/playground/12690252201/geometry -- http://www.w3.org/1999/02/22-rdf-syntax-ns#type --> http://www.opendatafithou.net/ont/geosparql#Point
5. http://opendatafithou.org/playground/691421446 -- http://www.opendatafithou.net/ont/geosparql#hasGeometry --> http://opendatafithou.org/playground/691421446/geometry
6. http://opendatafithou.org/playground/691421446/geometry -- http://www.opendatafithou.net/ont/geosparql#asWKT --> POINT(105.8352211 21.0213775)
7. http://opendatafithou.org/playground/899514990 -- http://www.

In [ ]:
from rdflib import Graph, Literal, RDF, URIRef, RDFS, Namespace

# Load file RDF hiện có
g = Graph()
g.parse("playgrounds.ttl", format="turtle")

# Namespace theo cấu hình của bạn
GEO = Namespace("http://www.opendatafithou.net/ont/geosparql#")
EX = Namespace("http://opendatafithou.org/playground/")
g.bind("geo", GEO)
g.bind("ex", EX)
g.bind("rdfs", RDFS)

# Liên kết Wikidata playground
WIKI = URIRef("https://www.wikidata.org/wiki/Q10793991")

# Mở rộng từng playground
for s in set(g.subjects(RDF.type, None)):
    # Thêm nguồn dữ liệu
    g.add((s, EX.source, Literal("OpenStreetMap")))

    # Liên kết Wikidata
    g.add((s, RDFS.seeAlso, WIKI))

    # Ví dụ thêm osm_type (nếu bạn có dữ liệu từ GeoJSON thì có thể lấy chính xác)
    g.add((s, EX.osmType, Literal("way")))

    # Ví dụ thêm tag leisure
    g.add((s, EX.leisure, Literal("playground")))

# Lưu file RDF mở rộng
expanded_file = "playgrounds_expanded.ttl"
g.serialize(expanded_file, format="turtle")
print(f"✅ Đã lưu RDF mở rộng: {expanded_file}")


✅ Đã lưu RDF mở rộng: playgrounds_expanded.ttl


In [ ]:
from rdflib import Graph

# Load file RDF vừa tạo
g = Graph()
g.parse("playgrounds_expanded.ttl", format="turtle")

# Hiển thị 10 triple đầu tiên
print("🔹 10 triple đầu tiên trong RDF:")
for i, (s, p, o) in enumerate(g):
    print(f"{i+1}. {s} -- {p} --> {o}")
    if i >= 9:
        break


🔹 10 triple đầu tiên trong RDF:
1. http://opendatafithou.org/playground/12050985881/geometry -- http://www.w3.org/2000/01/rdf-schema#seeAlso --> https://www.wikidata.org/wiki/Q10793991
2. http://opendatafithou.org/playground/1228194746/geometry -- http://www.opendatafithou.net/ont/geosparql#asWKT --> POINT(105.7398883 21.0044387)
3. http://opendatafithou.org/playground/1370518270/geometry -- http://www.w3.org/2000/01/rdf-schema#seeAlso --> https://www.wikidata.org/wiki/Q10793991
4. http://opendatafithou.org/playground/1218063064/geometry -- http://opendatafithou.org/playground/osmType --> way
5. http://opendatafithou.org/playground/1425341621/geometry -- http://opendatafithou.org/playground/leisure --> playground
6. http://opendatafithou.org/playground/199297711/geometry -- http://opendatafithou.org/playground/osmType --> way
7. http://opendatafithou.org/playground/10765202472 -- http://opendatafithou.org/playground/website --> https://www.sanchoi.org/
8. http://opendatafithou.org/play

In [ ]:
from rdflib import Graph, Literal, RDF, URIRef, Namespace, RDFS

# Load RDF cũ
g = Graph()
g.parse("playgrounds.ttl", format="turtle")

# Namespace theo cấu hình của bạn
GEO = Namespace("http://www.opendatafithou.net/ont/geosparql#")
EX = Namespace("http://opendatafithou.org/playground/")
g.bind("geo", GEO)
g.bind("ex", EX)
g.bind("rdfs", RDFS)

# URI Wikidata playground
WIKI = URIRef("https://www.wikidata.org/wiki/Q10793991")

# Lấy tất cả các playground (subject) dựa trên predicate geo:hasGeometry
playgrounds = set()
for s, p, o in g.triples((None, GEO.hasGeometry, None)):
    playgrounds.add(s)

# Mở rộng từng playground
for s in playgrounds:
    # Thêm metadata vào playground chính, không vào geometry
    g.add((s, EX.source, Literal("OpenStreetMap")))
    g.add((s, EX.osmType, Literal("way")))       # ví dụ default
    g.add((s, EX.leisure, Literal("playground"))) # ví dụ default
    g.add((s, RDFS.seeAlso, WIKI))

# Lưu file RDF sạch sẽ
expanded_file = "playgrounds_expanded_clean.ttl"
g.serialize(expanded_file, format="turtle")
print(f"✅ Đã lưu RDF mở rộng sạch sẽ: {expanded_file}")


✅ Đã lưu RDF mở rộng sạch sẽ: playgrounds_expanded_clean.ttl


In [ ]:
import os
import requests
import base64
import json

def get_github_file(file_path, save_as="tmp_file.geojson"):
    """
    Lấy nội dung file từ GitHub repo, lưu tạm file để dùng cho hàm chuyển RDF.
    Chỉ hiển thị số feature và tên file lưu tạm, không in toàn bộ dữ liệu.

    Args:
        file_path (str): Đường dẫn file trong repo, ví dụ "data/playground.geojson"
        save_as (str): Tên file tạm lưu trên Colab để dùng cho hàm RDF

    Returns:
        str | None: Tên file tạm, hoặc None nếu lỗi
    """
    url = f"https://api.github.com/repos/{GITHUB_USER}/{GITHUB_REPO}/contents/{file_path}?ref={BRANCH}"
    headers = {
        "Authorization": f"token {GITHUB_TOKEN}",
        "Accept": "application/vnd.github.v3+json"
    }

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        data = response.json()
        file_content = base64.b64decode(data['content']).decode('utf-8')
        geojson_data = json.loads(file_content)

        # Lưu tạm file
        with open(save_as, "w", encoding="utf-8") as f:
            json.dump(geojson_data, f, ensure_ascii=False, indent=2)

        # Hiển thị thông tin
        num_features = len(geojson_data.get("features", []))
        print(f"✅ Lấy thành công: {num_features} feature(s)")
        print(f"✅ File tạm lưu: {save_as}")

        return save_as
    else:
        print("❌ Lỗi:", response.status_code, response.text)
        return None

# Ví dụ sử dụng
tmp_file = get_github_file("data/playground.geojson")
if tmp_file:
    print(f"Bạn có thể truyền {tmp_file} vào hàm geojson_to_rdf_clean()")


✅ Lấy thành công: 136 feature(s)
✅ File tạm lưu: tmp_file.geojson
Bạn có thể truyền tmp_file.geojson vào hàm geojson_to_rdf_clean()


In [ ]:
get_github_file("data/atm.geojson")

✅ Lấy thành công: 1416 feature(s)
✅ File tạm lưu: tmp_file.geojson


'tmp_file.geojson'

In [ ]:
from rdflib import Graph, Literal, RDF, URIRef, Namespace, RDFS
import json

# Bản đồ OSM tag → Wikidata QID
WIKIDATA_MAP = {
    "playground": "Q10793991",
    "toilets": "Q813966",
    "atm": "Q12892",
    "bus_stop": "Q55488",
    "school": "Q3918",
    "hospital": "Q16917",
    "park": "Q22698"
}

def geojson_to_rdf_clean(file_geojson, output_file="poi_expanded_clean.ttl"):
    """
    Chuyển GeoJSON thành RDF Turtle mở rộng sạch sẽ, hỗ trợ nhiều loại POI.

    Args:
        file_geojson (str): Đường dẫn file GeoJSON đầu vào.
        output_file (str): File RDF Turtle đầu ra.
    """
    # Load GeoJSON
    with open(file_geojson, "r", encoding="utf-8") as f:
        geojson_data = json.load(f)

    # Khởi tạo RDF graph
    g = Graph()
    GEO = Namespace("http://www.opendatafithou.net/ont/geosparql#")
    EX = Namespace("http://opendatafithou.org/poi/")
    g.bind("geo", GEO)
    g.bind("ex", EX)
    g.bind("rdfs", RDFS)

    # URI mặc định cho seeAlso Wikidata khi có loại
    def get_wikidata_uri(poi_type):
        qid = WIKIDATA_MAP.get(poi_type)
        if qid:
            return URIRef(f"https://www.wikidata.org/wiki/{qid}")
        return None

    # Tạo RDF từ GeoJSON
    for feature in geojson_data.get("features", []):
        props = feature.get("properties", {})
        osm_id = props.get("osm_id")
        if not osm_id:
            continue
        subject = EX[str(osm_id)]

        # Label
        name = props.get("name") or props.get("tags", {}).get("name")
        if name:
            g.add((subject, RDFS.label, Literal(name)))

        # Các tag khác
        tags = props.get("tags", {})
        for k, v in tags.items():
            if k != "name":
                g.add((subject, EX[k], Literal(v)))

        # Geometry
        geom = feature.get("geometry")
        if geom:
            geom_uri = URIRef(str(subject) + "/geometry")
            g.add((geom_uri, RDF.type, GEO[geom["type"]]))
            coords = geom["coordinates"]
            if geom["type"] == "Point":
                wkt = f"POINT({coords[0]} {coords[1]})"
            g.add((geom_uri, GEO.asWKT, Literal(wkt)))
            g.add((subject, GEO.hasGeometry, geom_uri))

    # Mở rộng RDF sạch sẽ
    for s in set(g.subjects(RDF.type, None)):
        # Metadata mặc định
        g.add((s, EX.source, Literal("OpenStreetMap")))
        g.add((s, EX.osmType, Literal("way")))  # default
        # Xác định loại POI
        poi_type = None
        for p, o in g.predicate_objects(s):
            if p == EX.leisure or p == EX.amenity:
                poi_type = str(o)
                break
        # Nếu không có poi_type từ tag, mặc định playground
        if not poi_type:
            poi_type = "playground"
        g.add((s, EX.leisure, Literal(poi_type)))
        # Liên kết Wikidata nếu có
        wiki_uri = get_wikidata_uri(poi_type)
        if wiki_uri:
            g.add((s, RDFS.seeAlso, wiki_uri))

    # Lưu file RDF
    g.serialize(output_file, format="turtle")
    print(f"✅ Đã lưu RDF mở rộng sạch sẽ: {output_file}")
    return g



In [ ]:
geojson_to_rdf_clean("tmp_file.geojson", "atm_expanded_clean.ttl")

✅ Đã lưu RDF mở rộng sạch sẽ: atm_expanded_clean.ttl


<Graph identifier=N74b04305b631494da920912f6a095765 (<class 'rdflib.graph.Graph'>)>

In [ ]:
from rdflib import Graph

def show_rdf_triples(file_rdf, n=10, rdf_format="turtle"):
    """
    Hiển thị n triple đầu tiên của một file RDF.

    Args:
        file_rdf (str): Đường dẫn file RDF.
        n (int): Số triple muốn hiển thị.
        rdf_format (str): Format file RDF, mặc định "turtle".
    """
    g = Graph()
    g.parse(file_rdf, format=rdf_format)

    print(f"🔹 {n} triple đầu tiên trong RDF ({file_rdf}):")
    for i, (s, p, o) in enumerate(g):
        print(f"{i+1}. {s} -- {p} --> {o}")
        if i >= n-1:
            break



In [ ]:

# Ví dụ sử dụng
show_rdf_triples("atm_expanded_clean.ttl", n=10)

🔹 10 triple đầu tiên trong RDF (atm_expanded_clean.ttl):
1. http://opendatafithou.org/poi/724829588/geometry -- http://www.w3.org/2000/01/rdf-schema#seeAlso --> https://www.wikidata.org/wiki/Q10793991
2. http://opendatafithou.org/poi/1491048129 -- http://opendatafithou.org/poi/amenity --> atm
3. http://opendatafithou.org/poi/1491047983/geometry -- http://www.opendatafithou.net/ont/geosparql#asWKT --> POINT(105.8340583 20.9995766)
4. http://opendatafithou.org/poi/12896726155 -- http://opendatafithou.org/poi/operator --> TPBank
5. http://opendatafithou.org/poi/5125915206/geometry -- http://www.opendatafithou.net/ont/geosparql#asWKT --> POINT(105.8111519 21.0684335)
6. http://opendatafithou.org/poi/1498025790 -- http://opendatafithou.org/poi/official_name:vi --> Ngân hàng Đầu tư và Phát triển Việt Nam
7. http://opendatafithou.org/poi/9667843498/geometry -- http://opendatafithou.org/poi/leisure --> playground
8. http://opendatafithou.org/poi/12853640497 -- http://opendatafithou.org/poi/cas

In [ ]:
def push_file(file_local, repo_path, commit_message="Upload file via API"):
    """
    Đẩy file từ Colab/Local lên GitHub, sử dụng các cấu hình mặc định đã có.

    Args:
        file_local (str): File trên Colab/local.
        repo_path (str): Đường dẫn file trong repo, ví dụ "opendata_hanoi/playgrounds_expanded_clean.ttl".
        commit_message (str): Message commit.
    """
    if not os.environ.get('GITHUB_TOKEN'):
        print("❌ Lỗi: GITHUB_TOKEN chưa được thiết lập trong biến môi trường.")
        return None

    # Encode file thành base64
    with open(file_local, "rb") as f:
        content_b64 = base64.b64encode(f.read()).decode("utf-8")

    # Kiểm tra file đã tồn tại chưa
    check_url = f"https://api.github.com/repos/{GITHUB_USER}/{GITHUB_REPO}/contents/{repo_path}?ref={BRANCH}"
    headers = {
        "Authorization": f"token {GITHUB_TOKEN}",
        "Accept": "application/vnd.github.v3+json"
    }
    r = requests.get(check_url, headers=headers)
    sha = r.json().get("sha") if r.status_code == 200 else None

    # Payload upload/update
    data = {
        "message": commit_message,
        "content": content_b64,
        "branch": BRANCH
    }
    if sha:
        data["sha"] = sha  # cần nếu update file đã tồn tại

    # Đẩy file
    response = requests.put(check_url, headers=headers, data=json.dumps(data))
    if response.status_code in [200, 201]:
        print(f"✅ File {file_local} đã được đẩy lên GitHub: {repo_path}")
        return True
    else:
        print(f"❌ Lỗi đẩy file: {response.status_code} - {response.text}")
        return None



In [ ]:
# Ví dụ sử dụng
push_file("atm_expanded_clean.ttl", "opendata_hanoi/atm_expanded_clean.ttl", "Upload RDF atm file")

✅ File atm_expanded_clean.ttl đã được đẩy lên GitHub: opendata_hanoi/atm_expanded_clean.ttl


True

In [ ]:
get_github_file("data/drinking_water.geojson")

✅ Lấy thành công: 19 feature(s)
✅ File tạm lưu: tmp_file.geojson


'tmp_file.geojson'

In [ ]:
geojson_to_rdf_clean("tmp_file.geojson", "drinking_water_expanded_clean.ttl")

✅ Đã lưu RDF mở rộng sạch sẽ: drinking_water_expanded_clean.ttl


<Graph identifier=N4a784f6c45da4a0e85f4128d616d0bfd (<class 'rdflib.graph.Graph'>)>

In [ ]:
show_rdf_triples("drinking_water_expanded_clean.ttl", n=10)

🔹 10 triple đầu tiên trong RDF (drinking_water_expanded_clean.ttl):
1. http://opendatafithou.org/poi/9931507054 -- http://opendatafithou.org/poi/fountain --> bubbler
2. http://opendatafithou.org/poi/13005736901/geometry -- http://www.w3.org/1999/02/22-rdf-syntax-ns#type --> http://www.opendatafithou.net/ont/geosparql#Point
3. http://opendatafithou.org/poi/11834019569/geometry -- http://opendatafithou.org/poi/source --> OpenStreetMap
4. http://opendatafithou.org/poi/6698948887/geometry -- http://opendatafithou.org/poi/source --> OpenStreetMap
5. http://opendatafithou.org/poi/13005736901/geometry -- http://opendatafithou.org/poi/source --> OpenStreetMap
6. http://opendatafithou.org/poi/4758991221/geometry -- http://www.w3.org/1999/02/22-rdf-syntax-ns#type --> http://www.opendatafithou.net/ont/geosparql#Point
7. http://opendatafithou.org/poi/10114776470/geometry -- http://www.opendatafithou.net/ont/geosparql#asWKT --> POINT(105.8445302 21.0038661)
8. http://opendatafithou.org/poi/12690411

In [ ]:
push_file("drinking_water_expanded_clean.ttl", "opendata_hanoi/drinking_water_expanded_clean.ttl", "Upload RDF drinking_water file")

✅ File drinking_water_expanded_clean.ttl đã được đẩy lên GitHub: opendata_hanoi/drinking_water_expanded_clean.ttl


True

In [ ]:
get_github_file("data/hospital.geojson")

✅ Lấy thành công: 157 feature(s)
✅ File tạm lưu: tmp_file.geojson


'tmp_file.geojson'

In [ ]:
geojson_to_rdf_clean("tmp_file.geojson", "hospital_expanded_clean.ttl")

✅ Đã lưu RDF mở rộng sạch sẽ: hospital_expanded_clean.ttl


<Graph identifier=Ndd6bca23fed64f1ba79aa842f127a0ab (<class 'rdflib.graph.Graph'>)>

In [ ]:
show_rdf_triples("hospital_expanded_clean.ttl", n=10)

🔹 10 triple đầu tiên trong RDF (hospital_expanded_clean.ttl):
1. http://opendatafithou.org/poi/199148844 -- http://opendatafithou.org/poi/addr:province --> Hà Nội
2. http://opendatafithou.org/poi/12000027489 -- http://opendatafithou.org/poi/healthcare --> hospital
3. http://opendatafithou.org/poi/5839478946 -- http://opendatafithou.org/poi/addr:street --> Ngô Quyền
4. http://opendatafithou.org/poi/974596720/geometry -- http://www.w3.org/2000/01/rdf-schema#seeAlso --> https://www.wikidata.org/wiki/Q10793991
5. http://opendatafithou.org/poi/704147393 -- http://opendatafithou.org/poi/amenity --> hospital
6. http://opendatafithou.org/poi/704147393 -- http://www.opendatafithou.net/ont/geosparql#hasGeometry --> http://opendatafithou.org/poi/704147393/geometry
7. http://opendatafithou.org/poi/1165535247 -- http://opendatafithou.org/poi/operator:type --> government
8. http://opendatafithou.org/poi/384534300 -- http://www.opendatafithou.net/ont/geosparql#hasGeometry --> http://opendatafithou.or

In [ ]:
push_file("hospital_expanded_clean.ttl", "opendata_hanoi/hospital_expanded_clean.ttl", "Upload RDF hospital file")

✅ File hospital_expanded_clean.ttl đã được đẩy lên GitHub: opendata_hanoi/hospital_expanded_clean.ttl


True

In [ ]:
get_github_file("data/school.geojson")

✅ Lấy thành công: 1038 feature(s)
✅ File tạm lưu: tmp_file.geojson


'tmp_file.geojson'

In [ ]:
geojson_to_rdf_clean("tmp_file.geojson", "school_expanded_clean.ttl")

✅ Đã lưu RDF mở rộng sạch sẽ: school_expanded_clean.ttl


<Graph identifier=Nad58414186534b55b6b6a857d278a185 (<class 'rdflib.graph.Graph'>)>

In [ ]:
show_rdf_triples("school_expanded_clean.ttl", n=10)

🔹 10 triple đầu tiên trong RDF (school_expanded_clean.ttl):
1. http://opendatafithou.org/poi/1185911640/geometry -- http://www.w3.org/2000/01/rdf-schema#seeAlso --> https://www.wikidata.org/wiki/Q10793991
2. http://opendatafithou.org/poi/661138420 -- http://opendatafithou.org/poi/grades --> 1-5
3. http://opendatafithou.org/poi/810475989 -- http://opendatafithou.org/poi/addr:subdistrict --> Quang Trung
4. http://opendatafithou.org/poi/7409549825 -- http://opendatafithou.org/poi/addr:district --> Chương Mỹ
5. http://opendatafithou.org/poi/966204384 -- http://opendatafithou.org/poi/grades --> 1-5
6. http://opendatafithou.org/poi/875488227/geometry -- http://opendatafithou.org/poi/source --> OpenStreetMap
7. http://opendatafithou.org/poi/954924140/geometry -- http://opendatafithou.org/poi/leisure --> playground
8. http://opendatafithou.org/poi/659909921/geometry -- http://www.w3.org/2000/01/rdf-schema#seeAlso --> https://www.wikidata.org/wiki/Q10793991
9. http://opendatafithou.org/poi/4871

In [ ]:
push_file("school_expanded_clean.ttl", "opendata_hanoi/school_expanded_clean.ttl", "Upload RDF school file")

✅ File school_expanded_clean.ttl đã được đẩy lên GitHub: opendata_hanoi/school_expanded_clean.ttl


True

In [ ]:
get_github_file("data/toilets.geojson")

✅ Lấy thành công: 141 feature(s)
✅ File tạm lưu: tmp_file.geojson


'tmp_file.geojson'

In [ ]:
geojson_to_rdf_clean("tmp_file.geojson", "toilets_expanded_clean.ttl")

✅ Đã lưu RDF mở rộng sạch sẽ: toilets_expanded_clean.ttl


<Graph identifier=Nf3db29005bc643f9b7677071d932e9b1 (<class 'rdflib.graph.Graph'>)>

In [ ]:
show_rdf_triples("toilets_expanded_clean.ttl", n=10)

🔹 10 triple đầu tiên trong RDF (toilets_expanded_clean.ttl):
1. http://opendatafithou.org/poi/1228194740 -- http://opendatafithou.org/poi/building --> yes
2. http://opendatafithou.org/poi/1219283150/geometry -- http://www.w3.org/2000/01/rdf-schema#seeAlso --> https://www.wikidata.org/wiki/Q10793991
3. http://opendatafithou.org/poi/5304805821/geometry -- http://www.opendatafithou.net/ont/geosparql#asWKT --> POINT(105.8491565 21.02723)
4. http://opendatafithou.org/poi/1356130096 -- http://opendatafithou.org/poi/building --> yes
5. http://opendatafithou.org/poi/2587514016/geometry -- http://www.w3.org/2000/01/rdf-schema#seeAlso --> https://www.wikidata.org/wiki/Q10793991
6. http://opendatafithou.org/poi/1302250728 -- http://opendatafithou.org/poi/amenity --> toilets
7. http://opendatafithou.org/poi/1330108368/geometry -- http://www.w3.org/1999/02/22-rdf-syntax-ns#type --> http://www.opendatafithou.net/ont/geosparql#Point
8. http://opendatafithou.org/poi/11832128104/geometry -- http://open

In [ ]:
push_file("toilets_expanded_clean.ttl", "opendata_hanoi/toilets_expanded_clean.ttl", "Upload RDF toilets file")

✅ File toilets_expanded_clean.ttl đã được đẩy lên GitHub: opendata_hanoi/toilets_expanded_clean.ttl


True

In [ ]:
import requests
import json

# Đường dẫn raw file trên GitHub
url = "https://raw.githubusercontent.com/honganhss/OpenDataFitHou/refs/heads/main/data/bus_stop.geojson"

# Tải dữ liệu từ URL
response = requests.get(url)
if response.status_code == 200:
    try:
        # Đọc nội dung JSON
        geojson_data = response.json()

        # Lưu vào file tạm
        with open("tmp_bus_stop.geojson", "w", encoding="utf-8") as f:
            json.dump(geojson_data, f, ensure_ascii=False, indent=2)

        print("✅ Lưu thành công vào tmp_bus_stop.geojson")
    except json.JSONDecodeError:
        print("❌ Nội dung không phải JSON hợp lệ.")
else:
    print(f"❌ Không tải được file: {response.status_code}")


❌ Không tải được file: 404


In [ ]:
geojson_to_rdf_clean("tmp_bus_stop.geojson", "bus_stop_expanded_clean.ttl")

✅ Đã lưu RDF mở rộng sạch sẽ: bus_stop_expanded_clean.ttl


<Graph identifier=N405ce34e3f264b2da1bf346b56fe4342 (<class 'rdflib.graph.Graph'>)>

In [ ]:
show_rdf_triples("bus_stop_expanded_clean.ttl", n=10)

🔹 10 triple đầu tiên trong RDF (bus_stop_expanded_clean.ttl):
1. http://opendatafithou.org/poi/9342635870 -- http://www.opendatafithou.net/ont/geosparql#hasGeometry --> http://opendatafithou.org/poi/9342635870/geometry
2. http://opendatafithou.org/poi/8897455261 -- http://www.w3.org/2000/01/rdf-schema#label --> Trường CĐ Kỹ Thuật Công Nghiệp HN
3. http://opendatafithou.org/poi/8895739841/geometry -- http://opendatafithou.org/poi/osmType --> way
4. http://opendatafithou.org/poi/8846358782/geometry -- http://opendatafithou.org/poi/osmType --> way
5. http://opendatafithou.org/poi/8895558436/geometry -- http://www.w3.org/1999/02/22-rdf-syntax-ns#type --> http://www.opendatafithou.net/ont/geosparql#Point
6. http://opendatafithou.org/poi/9019312710/geometry -- http://www.opendatafithou.net/ont/geosparql#asWKT --> POINT(105.779622 20.9882876)
7. http://opendatafithou.org/poi/8919736492/geometry -- http://opendatafithou.org/poi/osmType --> way
8. http://opendatafithou.org/poi/738813822 -- http

In [ ]:
push_file("bus_stop_expanded_clean.ttl", "opendata_hanoi/bus_stop_expanded_clean.ttl", "Upload RDF bus_stop file")

✅ File bus_stop_expanded_clean.ttl đã được đẩy lên GitHub: opendata_hanoi/bus_stop_expanded_clean.ttl


True